In [1]:
import cv2
import Tkinter as tk
import tkFileDialog
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from Preprocessing import load_images
from keras.utils import np_utils
%matplotlib inline

Using TensorFlow backend.


In [2]:
tk.Tk().withdraw()
path_Train=tkFileDialog.askdirectory()
print path_Train

/home/edward/project /Dataset/Training Images


In [3]:
X_Train=load_images(path_Train)

In [4]:
X_Train=np.array(X_Train)
X_Train.shape

(2000, 64, 64, 3)

In [5]:
X_Train=np.reshape(X_Train,[-1,np.prod(X_Train.shape[1:])])
X_Train.shape

(2000, 12288)

In [6]:
tk.Tk().withdraw()
path_Validation=tkFileDialog.askdirectory()
print path_Validation

/home/edward/project /Dataset/Validation Images


In [7]:
X_Test=load_images(path_Validation)
X_Test=np.array(X_Test)
X_Test.shape

(150, 64, 64, 3)

In [8]:
X_Test=np.reshape(X_Test,[-1,np.prod(X_Test.shape[1:])])
X_Test.shape

(150, 12288)

In [9]:
data_Train=pd.read_csv("/home/edward/Skin Cancer Data/ISIC-2017_Training_Part3_GroundTruth.csv")
data_Test=pd.read_csv("/home/edward/Skin Cancer Data/ISIC-2017_Validation_Part3_GroundTruth.csv")

In [10]:
data_Train=data_Train.iloc[0:2000,1]
data_Test=data_Test.iloc[0:150,1]
print data_Train.shape
print data_Test.shape

(2000,)
(150,)


In [11]:
y_Train=data_Train
y_test=data_Test

In [12]:
y_Train=np.array(y_Train)
y_Test=np.array(y_test)

In [13]:
y_Train = np_utils.to_categorical(data_Train)
y_Test = np_utils.to_categorical(data_Test)
print y_Train.shape
print y_Test.shape

(2000, 2)
(150, 2)


In [14]:
learning_rate=0.001
training_epoch=15
batch_size=100
display_step=1
logs_path = '/tmp/tensorflow_logs/example'
n_hidden_1=256
n_hidden_2=256
#n_hidden_3=256
#n_hidden_4=256
n_input=64*64*3
n_classes=2

x=tf.placeholder("float",[None,n_input])
y=tf.placeholder("float",[None,n_classes])

In [15]:
def multiplayer_perceptron(x,weights,biases):
    # first layer
    #with tf.device('/gpu:0'):
    #x=tf.reshape(x,shape=[-1,64,64,1])
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.sigmoid(layer_1)
    # second layer
    #with tf.device('/gpu:1'):
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.sigmoid(layer_2)
    out_layer=tf.matmul(layer_2,weights['out'])+biases['out']
    return out_layer

In [16]:
# store layer weights and biases
weights={
    'h1':tf.Variable(tf.random_normal([n_input,n_hidden_1])),
    'h2':tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    #'h3':tf.Variable(tf.random_normal([n_hidden_2,n_hidden_3])),
    #'h4':tf.Variable(tf.random_normal([n_hidden_3,n_hidden_4])),
    'out':tf.Variable(tf.random_normal([n_hidden_1,n_classes]))
}

biases={
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    #'b3':tf.Variable(tf.random_normal([n_hidden_3])),
    #'b4':tf.Variable(tf.random_normal([n_hidden_4])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}
with tf.name_scope("Model"):
    pred=multiplayer_perceptron(x,weights,biases)

# define loss and optimizer
with tf.name_scope('Loss'):
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
with tf.name_scope('Adam'):
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
with tf.name_scope("Accuracy"):
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
# initilizing the variables
init=tf.global_variables_initializer()

tf.summary.scalar("loss",cost)
tf.summary.scalar("accuracy",acc)
merged_summary_op=tf.summary.merge_all()

In [20]:
from sklearn import metrics as sk
# launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    summary_writer=tf.summary.FileWriter(logs_path,graph=sess.graph)
    # Training 
    
    for epoch in range(training_epoch):
        avg_cost=0
        for i in range(batch_size):
            train_idx=np.random.randint(X_Train.shape[0],size=100)
            batch_x_Train=X_Train[train_idx,:]
            batch_y_Train=y_Train[train_idx]
            _,c,summary=sess.run([optimizer,cost,merged_summary_op],feed_dict={x:batch_x_Train,y:batch_y_Train})
            summary_writer.add_summary(summary,epoch*batch_size+i)
            avg_cost+=c/batch_size
            
        if epoch%display_step==0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished"
    
    Predictions=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    accuracy=tf.reduce_mean(tf.cast(Predictions,"float"))
    test_idx=np.random.randint(X_Test.shape[0],size=100)
    batch_x_Test=X_Test[test_idx,:]
    batch_y_Test=y_Test[test_idx]
    print "Training Accuracy:" ,accuracy.eval({x:batch_x_Train,y:batch_y_Train})
    print "Testing Accuracy:" ,accuracy.eval({x:batch_x_Test,y:batch_y_Test})
    y_p = tf.argmax(pred, 1)
    val_accuracy, y_pred = sess.run([accuracy, y_p], feed_dict={x:X_Test, y:y_Test})

    print "validation accuracy:", val_accuracy
    y_true = np.argmax(y_Test,1)
    print "Precision", sk.precision_score(y_true, y_pred)
    print "Recall", sk.recall_score(y_true, y_pred)
    print "f1_score", sk.f1_score(y_true, y_pred)
    print "confusion_matrix"
    print sk.confusion_matrix(y_true, y_pred)
    fpr, tpr, tresholds = sk.roc_curve(y_true, y_pred)

Epoch: 0001 cost= 0.848674871
Epoch: 0002 cost= 0.486426866
Epoch: 0003 cost= 0.479657972
Epoch: 0004 cost= 0.458090131
Epoch: 0005 cost= 0.452489701
Epoch: 0006 cost= 0.448284647
Epoch: 0007 cost= 0.437242183
Epoch: 0008 cost= 0.434923283
Epoch: 0009 cost= 0.427488716
Epoch: 0010 cost= 0.429780490
Epoch: 0011 cost= 0.434446906
Epoch: 0012 cost= 0.424893203
Epoch: 0013 cost= 0.437195039
Epoch: 0014 cost= 0.421536350
Epoch: 0015 cost= 0.430040421
Optimization Finished
Training Accuracy: 0.82
Testing Accuracy: 0.83
validation accuracy: 0.806667
Precision 1.0
Recall 0.0333333333333
f1_score 0.0645161290323
confusion_matrix
[[120   0]
 [ 29   1]]


In [ ]:
with tf.Session() as sess:
    y_p = tf.argmax(pred, 1)
    val_accuracy, y_pred = sess.run([accuracy, y_p], feed_dict={x:X_Test, y:y_Test})

    print "validation accuracy:", val_accuracy
    y_true = np.argmax(y_Test,1)
    print "Precision", sk.metrics.precision_score(y_true, y_pred)
    print "Recall", sk.metrics.recall_score(y_true, y_pred)
    print "f1_score", sk.metrics.f1_score(y_true, y_pred)
    print "confusion_matrix"
    print sk.metrics.confusion_matrix(y_true, y_pred)
    fpr, tpr, tresholds = sk.metrics.roc_curve(y_true, y_pred)